# Learnt biposelet visualisation

This notebook looks at the training HDF5 file and, for each biposelet, scrapes out some relevant examples. The resultant figures should be helpful for my paper.

In [ ]:
%matplotlib inline

from os import path, makedirs

import h5py

import numpy as np

import matplotlib.pyplot as plt

from scipy.misc import imsave

from train import infer_sizes

In [ ]:
train_h5_path = '../cache/mpii-cooking/train-patches/samples-000001.h5'
train_h5 = h5py.File(train_h5_path, 'r')
train_images = train_h5['images']
poselets = train_h5['poselet']
poselets_flat = np.argmax(poselets, 1)
ds_shape = infer_sizes(train_h5_path)

In [ ]:
np.random.seed(229) # Determinism
samples_per_poselet = 10
poselets_to_choose = 10
dest_path = '/home/sam/delete-me/poselet-scrapes/'
all_poselet_nums = np.arange(1, poselets.shape[1])
np.random.shuffle(all_poselet_nums)
chosen_poselet_nums = all_poselet_nums[:poselets_to_choose]

In [ ]:
def get_samples_for_poselet(poselet, samples_per_poselet=samples_per_poselet, 
                            all_images=train_images, all_poselets=poselets_flat):
    valid_samples, = np.nonzero(all_poselets == poselet)
    np.random.shuffle(valid_samples)
    valid_samples = valid_samples[:samples_per_poselet]
    assert len(valid_samples) == samples_per_poselet
    # bin_mask = np.full((len(all_poselets),), False, dtype=bool)
    # bin_mask[valid_samples] = True
    h5_indices = sorted(valid_samples.tolist())
    images = np.transpose(all_images[h5_indices], axes=[0, 2, 3, 1])
    images = images.reshape(images.shape[:-1] + (2, 3)) \
        .transpose([0, 3, 1, 2, 4])
    return images

In [ ]:
p1_samples = get_samples_for_poselet(256)
fig, subplots = plt.subplots(2, len(p1_samples))
for idx, frames in enumerate(p1_samples):
    for f in range(2):
        sp = subplots[f][idx]
        im = frames[f]
        sp.imshow(im)
        sp.axis('off')
fig.set_size_inches((14, 3))
plt.show()

In [ ]:
for chosen_num in chosen_poselet_nums:
    samples = get_samples_for_poselet(chosen_num)
    save_dir = path.join(dest_path, 'poselet-%i' % chosen_num)
    try:
        makedirs(save_dir)
    except OSError:
        pass
    for idx, frames in enumerate(samples):
        for f in range(2):
            im = frames[f]
            save_path = path.join(save_dir, 'sample-%i-f%i.jpg' % (idx, f))
            print('Saving image to %s' % save_path)
            imsave(save_path, im)